# Import and Manipulate Data:

Stock Data: [Link](https://www.kaggle.com/datasets/kalilurrahman/nasdaq100-stock-price-data?resource=download)
Index data was downloaded from Yahoo Finance.

The index is the Nasdaq (QQQ) and the individual stocks are the 102 stocks in the Nasdaq index.

Note: In the calculation functions (see functions.py) only NumPy is used, but here other functions are used for ease of use.

In [1]:
import numpy as np
import pandas as pd
import timeit
import functions as f

In [2]:
stocks = pd.read_csv('Nasdaq_100.csv', sep='\t')
stocks['Date'] = pd.to_datetime(stocks['Date']).dropna(axis=0)
stocks['Return'] = (stocks['Close'] - stocks['Open']) / stocks['Open']
stocks.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Name,Return
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.562591,493729600,AAPL,0.002718
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.573935,601904800,AAPL,-0.001025
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.469369,552160000,AAPL,-0.015906
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.457407,477131200,AAPL,-0.005525
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.500339,447610800,AAPL,0.007989


In [3]:
index = pd.read_csv('QQQ.csv')
index['Date'] = pd.to_datetime(index['Date'])
index['Return'] = (index['Close'] - index['Open']) / index['Open']
index.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Return
0,2010-01-04,46.330002,46.490002,46.270000,46.419998,41.241234,62822800,0.001942
1,2010-01-05,46.389999,46.500000,46.160000,46.419998,41.241234,62935600,0.000647
2,2010-01-06,46.400002,46.549999,46.070000,46.139999,40.992489,96033000,-0.005604
3,2010-01-07,46.209999,46.270000,45.919998,46.169998,41.019131,77094100,-0.000866
4,2010-01-08,46.070000,46.549999,45.930000,46.549999,41.356743,88886600,0.010419


In [4]:
stocks = stocks.pivot(columns='Name', index='Date', values='Return')[:-1].dropna(axis=0).swapaxes("index", "columns")
stock_tickers = stocks.index.values.tolist()
stocks.head()

Date,2019-09-26,2019-09-27,2019-09-30,2019-10-01,2019-10-02,2019-10-03,2019-10-04,2019-10-07,2019-10-08,2019-10-09,...,2021-08-26,2021-08-27,2021-08-30,2021-08-31,2021-09-01,2021-09-02,2021-09-03,2021-09-07,2021-09-08,2021-09-09
Name,,,,,,,,,,,,,,,,,,,,,
AAPL,-0.000500,-0.007799,0.013898,-0.002133,-0.018381,0.010942,0.006072,0.003491,-0.006288,0.000000,...,-0.005460,0.007594,0.027651,-0.005437,-0.002094,-0.001430,0.003512,0.011099,-0.011912,-0.009132
ADBE,0.019213,-0.024910,-0.004648,-0.022510,-0.016668,0.020530,0.014097,0.002099,-0.013226,0.002742,...,-0.001836,0.009195,0.009412,-0.006288,0.002077,-0.007733,0.003251,-0.008931,0.004650,-0.004753
ADI,0.002570,-0.016997,-0.003745,-0.043812,-0.001466,0.018371,0.010550,-0.018420,-0.035094,0.002784,...,0.014303,-0.004901,0.000060,-0.019260,-0.016242,0.003022,-0.000492,0.003750,-0.000244,0.033931
ADP,0.006349,-0.020514,0.003668,-0.011583,-0.014783,0.007547,0.006577,-0.000377,-0.007807,0.005202,...,-0.003321,0.006503,-0.000096,-0.004002,-0.003084,0.005061,0.014083,-0.022382,0.012138,-0.007814
ADSK,0.015048,-0.027984,0.013449,-0.018788,-0.007987,0.016858,0.006923,0.010823,-0.003558,0.007127,...,-0.022100,0.004487,-0.011201,-0.009708,-0.027453,-0.062088,0.007150,-0.011350,0.010857,-0.007350


In [5]:
index_ar = index['Return'].to_numpy()[-493:]
time_vector = index['Date'].to_numpy()[-473:]
stocks_ar = stocks.to_numpy()

# First Cut Beta Calculation

Iterates through each stock, then each time window within the stock to calculate beta using the formula:

${\beta}=\dfrac{\sum_{i=1}^{n}\left(x_{i}-\bar{x}\right)\left(y_{i}-\bar{y}\right)}{\sum_{i=1}^{n}\left(x_{i}-\bar{x}\right)^{2}}$
where $x$ is each sample return of the index (QQQ) and $y$ is the sample return of the individual stock, each over the window length $n$.

This method is extremely inefficient, taking 1.9745448000030592 seconds.

In [6]:
start = timeit.default_timer()
beta_ar = f.calc_beta_naive(20, stocks_ar, index_ar)
stop = timeit.default_timer()
print('Time Taken: ', stop - start)

Time Taken:  1.9745448000030592


# Optimized Beta Calculation

Selects entire matrix as a sample $Y$, with stock as the columns and each individual sample as a row, resulting in a $(n, 102)$ sized matrix where $n$ is the window length. Selects $X$ as a matrix with first column vector of all ones and second column vector of $n$ samples of index returns, resulting in a $(n, 2)$ matrix. Then uses the formula:
$\hat{\boldsymbol{\beta}}=\left(\mathbf{X}^{\top} \mathbf{X}\right)^{-\mathbf{1}} \mathbf{X}^{\top} \mathbf{Y}$ where $\hat{\boldsymbol{\beta}}=\begin{bmatrix}\alpha \\ \beta \end{bmatrix}$ Thus $\hat{\boldsymbol{\beta}}[1]$ is our beta. Iterates over whole array to calculate all beta values.

This method is significantly more efficient than the previous only taking 0.05131219999748282 seconds.

In [7]:
stocks_ar2 = stocks_ar.swapaxes(0, 1)

In [8]:
start = timeit.default_timer()
beta_ar2 = f.calc_beta(20, stocks_ar2, index_ar)
stop = timeit.default_timer()
print('Time Taken: ', stop - start)

Time Taken:  0.05131219999748282


# Resulting betas (in order):

In [9]:
result1 = pd.DataFrame(beta_ar, columns=stock_tickers)
result1['Date'] = time_vector
result1.set_index('Date', inplace=True)
result1.to_csv('result1.csv')
result1.head()

,AAPL,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,AMGN,...,TSLA,TXN,VRSK,VRSN,VRTX,WBA,WDAY,XEL,XLNX,ZM
Date,,,,,,,,,,,,,,,,,,,,,
2019-10-24,0.683423,1.499121,1.428348,1.080093,1.162515,0.298545,0.818730,1.139876,1.000031,0.571063,...,-0.216832,1.032757,1.271839,1.549078,1.028796,0.637227,1.745512,0.491763,0.709564,1.737399
2019-10-25,0.657976,1.589301,1.417874,1.116175,1.231752,0.330161,0.880601,1.248967,0.989307,0.547117,...,-0.152174,1.020892,1.304210,1.609413,1.028594,0.607869,1.850071,0.478759,0.741682,1.823384
2019-10-28,0.675917,1.504004,1.505396,0.907603,0.986388,0.101581,0.240352,1.015470,1.049239,0.524939,...,0.844589,1.029742,1.024349,0.908688,0.772672,0.904103,1.720181,0.288491,0.892279,1.226913
2019-10-29,0.646587,1.522459,1.563432,0.891875,0.933166,0.095068,0.253630,1.069239,1.099909,0.546256,...,0.886244,0.948140,0.948151,0.909988,0.767613,0.837089,1.654258,0.248027,0.865168,1.382549
2019-10-30,0.837578,1.414812,1.275584,0.837900,0.870544,0.089044,0.115909,1.219332,1.217743,0.509717,...,1.179153,0.933358,0.818527,0.821446,0.573677,0.705079,1.819548,0.322450,0.852061,1.468100


In [10]:
result2 = pd.DataFrame(beta_ar2, columns=stock_tickers)
result2['Date'] = time_vector
result2.set_index('Date', inplace=True)
result2.to_csv('result2.csv')
result2.head()

,AAPL,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,AMGN,...,TSLA,TXN,VRSK,VRSN,VRTX,WBA,WDAY,XEL,XLNX,ZM
Date,,,,,,,,,,,,,,,,,,,,,
2019-10-24,0.683423,1.499121,1.428348,1.080093,1.162515,0.298545,0.818730,1.139876,1.000031,0.571063,...,-0.216832,1.032757,1.271839,1.549078,1.028796,0.637227,1.745512,0.491763,0.709564,1.737399
2019-10-25,0.657976,1.589301,1.417874,1.116175,1.231752,0.330161,0.880601,1.248967,0.989307,0.547117,...,-0.152174,1.020892,1.304210,1.609413,1.028594,0.607869,1.850071,0.478759,0.741682,1.823384
2019-10-28,0.675917,1.504004,1.505396,0.907603,0.986388,0.101581,0.240352,1.015470,1.049239,0.524939,...,0.844589,1.029742,1.024349,0.908688,0.772672,0.904103,1.720181,0.288491,0.892279,1.226913
2019-10-29,0.646587,1.522459,1.563432,0.891875,0.933166,0.095068,0.253630,1.069239,1.099909,0.546256,...,0.886244,0.948140,0.948151,0.909988,0.767613,0.837089,1.654258,0.248027,0.865168,1.382549
2019-10-30,0.837578,1.414812,1.275584,0.837900,0.870544,0.089044,0.115909,1.219332,1.217743,0.509717,...,1.179153,0.933358,0.818527,0.821446,0.573677,0.705079,1.819548,0.322450,0.852061,1.468100
